In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import ast
import sklearn
from collections import Counter
from tqdm.notebook import trange, tqdm

# Load the labeled comments

In [2]:
# pd.read_csv('./data/both_sheet.csv').Sentimen.value_counts()
len(pd.read_csv('./data/both_sheet.csv'))

1460

In [3]:
fulldf = pd.read_csv('./data/both_sheet.csv')
fulldf = fulldf[~fulldf.Sentimen.isna()].reset_index(drop=True)
fulldf[fulldf.text_display.str.contains('Mudah')]

,id_komentar,author_name,text_display,Sentimen
11,UgxIYgX15ktKkPHD_Jp4AaABAg,Dono Bonek,"Mudah""Han dengan adanya new normal masyarakat ...",1.0
117,UgzPhA5bbqdqFSRnsJ14AaABAg,rita emil,"Anak sekolah mulai masuk juga ya, sempet was ...",0.0
166,UgwmIetxeEQ52ZAk8qt4AaABAg,suratno suratno,new normal / hidup berdampingan/ berdamai deng...,0.0
196,UgzZ_9wGYyxHqt0rgP14AaABAg,Sola Gaming,Pak Menurut Saya Mending Biarkan Covid 19 Dibi...,-1.0
789,UgyQ0ia5pgwlRFPNARh4AaABAg,Riady Tandy,"jga diri kita masing2, ikut anjuran pemerintah...",1.0
829,Ugwqf5vUXsxJtQmbUrJ4AaABAg,Andry Sacho,Mudah mudahan warga china di larang selamanya ...,0.0
934,UgyVKn6x_U0VJsH9C594AaABAg,fachri yansyah,Mudah-mudahan di tempat ibadah & sekolah sdh n...,1.0
1033,Ugz7D1-87joN1zD7YdR4AaABAg,Abu Syuroih,Mudah mudahan allah memudahkan urusan kita...,0.0
1190,UgzUUTPTSRE-9PzmPCd4AaABAg,Patah Kayu,Mudahan yang nonton video ini bersyukur akan k...,0.0


### Ambil kolom text and sentimen 

In [4]:
# df = fulldf[['id_komentar']].copy()
df = pd.DataFrame()
df['text'] = fulldf['text_display'].astype(str)
df['sentimen'] = fulldf['Sentimen'].astype('int')
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1460 non-null   object
 1   sentimen  1460 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 17.2+ KB


In [5]:
df.head(5)

,text,sentimen
0,Ya allah musnahkan lah virus corona dr muka bu...,0
1,"sekarang saya malu , kenapa dulu terlalu memb...",-1
2,"Peraturan anehh semua ,,, aku nyaris mati kela...",-1
3,Saya selalu nunggu new normal biar saya segera...,1
4,Semoga covid cepat berlallu..amin🤲 \nKaltim pu...,1


# Punctuation

In [6]:
df['text'] = df["text"].str.lower()

In [7]:
# Removal of Punctuations

col_from = "text"
col_save = "text_wo_punct"
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
#     return text.translate(str.maketrans(' ', ' ', PUNCT_TO_REMOVE))
#     return text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    # replace punc with space
    text = text.replace('\n', ' ')
#     text = text.replace('"', ' ')
#     text = text.replace("'", ' ')
    return text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    return text

df[col_save] = df[col_from].apply(lambda text: remove_punctuation(text))
COL = col_save
df.head()

,text,sentimen,text_wo_punct
0,ya allah musnahkan lah virus corona dr muka bu...,0,ya allah musnahkan lah virus corona dr muka bu...
1,"sekarang saya malu , kenapa dulu terlalu memb...",-1,sekarang saya malu kenapa dulu terlalu memb...
2,"peraturan anehh semua ,,, aku nyaris mati kela...",-1,peraturan anehh semua aku nyaris mati kela...
3,saya selalu nunggu new normal biar saya segera...,1,saya selalu nunggu new normal biar saya segera...
4,semoga covid cepat berlallu..amin🤲 \nkaltim pu...,1,semoga covid cepat berlallu amin🤲 kaltim pun...


# Hapus 
 - 'wkwkkw', 
 - '\n', 
 - huruf >1 di akhir kata, 'aminnn'->'amin', 'apaa'->'apa', 'iyaaaa'->'iya'
 - hurut >2 di kata, 'amiiiin'->'amin', 'sekaraaang'->'sekarang'

In [8]:
col_from = "text_wo_punct"
# col_from = "text"
col_save = "text_custom"
def remove_(text):
    # remove wkwk
    text = re.sub(r'\b(?=\w*wk)\w+', '', text)
    # remove DIGIT
    text = re.sub(r'([0-9])+', '', text)
    # Hapus huruf >1 di akhir kata
    text = re.sub(r'(.)\1{1,}\b', r'\1', text)
    # Hapus huruf >2 di kata
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    # Hapus newline
    text = text.replace('\n', '')
    return text

df[col_save] = df[col_from].apply(lambda text: remove_(text))
COL = col_save
df.head(5)

,text,sentimen,text_wo_punct,text_custom
0,ya allah musnahkan lah virus corona dr muka bu...,0,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dr muka bu...
1,"sekarang saya malu , kenapa dulu terlalu memb...",-1,sekarang saya malu kenapa dulu terlalu memb...,sekarang saya malu kenapa dulu terlalu membang...
2,"peraturan anehh semua ,,, aku nyaris mati kela...",-1,peraturan anehh semua aku nyaris mati kela...,peraturan aneh semua aku nyaris mati kelapan s...
3,saya selalu nunggu new normal biar saya segera...,1,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...
4,semoga covid cepat berlallu..amin🤲 \nkaltim pu...,1,semoga covid cepat berlallu amin🤲 kaltim pun...,semoga covid cepat berlallu amin🤲 kaltim pun k...


# Hapus emoji 

In [9]:
# REPLACE EMOJIS
col_from = "text_custom"
col_save = "text_custom"

def getEmojis():
    emojis = {}
    dfemo = pd.read_csv('data\\emoji_df.csv')
    emokeys = dfemo.emoji.values
    emoval = dfemo.codepoints
    for i in range(len(emokeys)):
        emojis[emokeys[i]] = emoval[i]
    return emojis
EMOJIS = getEmojis()

def convert_emojis(text):
    for emot in EMOJIS:
        text = str.replace(text, emot, ' '+EMOJIS[emot]+' ')
    return text
def remove_(text, pbar):
    """custom function to remove ..."""
    pbar.update(1)
    return convert_emojis(text)

with tqdm(total=len(df), desc="replacing emojis...") as pbar:
    df[col_save] = df[col_from].apply(lambda text: remove_(text, pbar))
COL = col_save
df.head(5)

,text,sentimen,text_wo_punct,text_custom
0,ya allah musnahkan lah virus corona dr muka bu...,0,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dr muka bu...
1,"sekarang saya malu , kenapa dulu terlalu memb...",-1,sekarang saya malu kenapa dulu terlalu memb...,sekarang saya malu kenapa dulu terlalu membang...
2,"peraturan anehh semua ,,, aku nyaris mati kela...",-1,peraturan anehh semua aku nyaris mati kela...,peraturan aneh semua aku nyaris mati kelapan s...
3,saya selalu nunggu new normal biar saya segera...,1,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...
4,semoga covid cepat berlallu..amin🤲 \nkaltim pu...,1,semoga covid cepat berlallu amin🤲 kaltim pun...,semoga covid cepat berlallu amin 1F932 kaltim...


# Ganti kata singkatan
 - 'yg -> 'yang'
 - 'smg' -> 'semoga'
 - 'tdk', 'g', 'ga', 'ngga', dll -> 'tidak'
 - dll

In [10]:
singkatan = {'find':[], 'replace':[]}
with open('data\\singkat.txt') as f:
    for t in f.read().split('\n'):
        singkatan['find'].append(t.split(',')[0].strip())
        singkatan['replace'].append(t.split(',')[1].strip())
print(singkatan['find'].index('yg'))

95


In [11]:
# Replace kata yang disingkat
col_from = "text_custom"
col_save = "text_rmvcustom"
def remove_(text):
    """custom function to remove the """
    for i in range(len(singkatan['replace'])):
#         if re.search(r'\b('+singkatan['find'][i]+r')\b', text):
#             print(text)
        text = re.sub(r'\b('+singkatan['find'][i]+r')\b', ' '+singkatan['replace'][i]+' ', text)
    return text

df[col_save] = df[col_from].apply(lambda text: remove_(text))
COL = col_save
df.head(5)

,text,sentimen,text_wo_punct,text_custom,text_rmvcustom
0,ya allah musnahkan lah virus corona dr muka bu...,0,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dari muk...
1,"sekarang saya malu , kenapa dulu terlalu memb...",-1,sekarang saya malu kenapa dulu terlalu memb...,sekarang saya malu kenapa dulu terlalu membang...,sekarang saya malu kenapa dulu terlalu membang...
2,"peraturan anehh semua ,,, aku nyaris mati kela...",-1,peraturan anehh semua aku nyaris mati kela...,peraturan aneh semua aku nyaris mati kelapan s...,peraturan aneh semua saya nyaris mati kelapa...
3,saya selalu nunggu new normal biar saya segera...,1,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...
4,semoga covid cepat berlallu..amin🤲 \nkaltim pu...,1,semoga covid cepat berlallu amin🤲 kaltim pun...,semoga covid cepat berlallu amin 1F932 kaltim...,semoga covid cepat berlallu amin 1F932 kaltim...


# Stopwords

In [14]:
# Stopwords removal
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
 
factory = StopWordRemoverFactory()
# stopword = factory.create_stop_word_remover()
# print(factory.get_stop_words())

# STOPWORDS = set(stopwords.words('indonesian'))
STOPWORDS = factory.get_stop_words()
with open('data\\tambahan_stopwords.txt') as f:
    for t in re.sub(r"[\',]", '', f.read()).split():
        STOPWORDS.append(t) 
print(STOPWORDS)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [15]:
STOPWORDS.remove('kenapa')
STOPWORDS.remove('apakah')
# STOPWORDS.remove('apa')

In [16]:
col_from = "text_rmvcustom"
col_save = "text_wo_stop"
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split(' ') if word not in STOPWORDS])

df[col_save] = df[col_from].apply(lambda text: remove_stopwords(text))
COL = col_save
df.head()

,text,sentimen,text_wo_punct,text_custom,text_rmvcustom,text_wo_stop
0,ya allah musnahkan lah virus corona dr muka bu...,0,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dari muk...,allah musnahkan virus corona muka bumi menja...
1,"sekarang saya malu , kenapa dulu terlalu memb...",-1,sekarang saya malu kenapa dulu terlalu memb...,sekarang saya malu kenapa dulu terlalu membang...,sekarang saya malu kenapa dulu terlalu membang...,sekarang malu kenapa dulu terlalu membanggakan...
2,"peraturan anehh semua ,,, aku nyaris mati kela...",-1,peraturan anehh semua aku nyaris mati kela...,peraturan aneh semua aku nyaris mati kelapan s...,peraturan aneh semua saya nyaris mati kelapa...,peraturan aneh semua nyaris mati kelapan sam...
3,saya selalu nunggu new normal biar saya segera...,1,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...,selalu nunggu new normal segera melangsungkan ...
4,semoga covid cepat berlallu..amin🤲 \nkaltim pu...,1,semoga covid cepat berlallu amin🤲 kaltim pun...,semoga covid cepat berlallu amin 1F932 kaltim...,semoga covid cepat berlallu amin 1F932 kaltim...,semoga covid cepat berlallu amin 1F932 kaltim...


# Stemming

In [17]:
#Stemming
# from tqdm.notebook import trange, tqdm
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
col_from = "text_wo_stop"
col_save = "text_stemmed"
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stem_words(text, pbar):
    pbar.update(1)
    return " ".join([stemmer.stem(word) for word in text.split(' ')])

with tqdm(total=len(df), desc="stemming...") as pbar:
    df[col_save] = df[col_from].apply(lambda text: stem_words(text, pbar))
COL = col_save
df.head()

,text,sentimen,text_wo_punct,text_custom,text_rmvcustom,text_wo_stop,text_stemmed
0,ya allah musnahkan lah virus corona dr muka bu...,0,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dari muk...,allah musnahkan virus corona muka bumi menja...,allah musnah virus corona muka bumi jalan hi...
1,"sekarang saya malu , kenapa dulu terlalu memb...",-1,sekarang saya malu kenapa dulu terlalu memb...,sekarang saya malu kenapa dulu terlalu membang...,sekarang saya malu kenapa dulu terlalu membang...,sekarang malu kenapa dulu terlalu membanggakan...,sekarang malu kenapa dulu terlalu bangga belia...
2,"peraturan anehh semua ,,, aku nyaris mati kela...",-1,peraturan anehh semua aku nyaris mati kela...,peraturan aneh semua aku nyaris mati kelapan s...,peraturan aneh semua saya nyaris mati kelapa...,peraturan aneh semua nyaris mati kelapan sam...,atur aneh semua nyaris mati kelap sama anak ...
3,saya selalu nunggu new normal biar saya segera...,1,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...,selalu nunggu new normal segera melangsungkan ...,selalu nunggu new normal segera langsung nikah...
4,semoga covid cepat berlallu..amin🤲 \nkaltim pu...,1,semoga covid cepat berlallu amin🤲 kaltim pun...,semoga covid cepat berlallu amin 1F932 kaltim...,semoga covid cepat berlallu amin 1F932 kaltim...,semoga covid cepat berlallu amin 1F932 kaltim...,moga covid cepat berlallu amin 1f932 kaltim s...


# Most freq words

In [18]:
from collections import Counter
cnt = Counter()
for text in df["text_stemmed"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(5)

[('normal', 484),
 ('new', 408),
 ('rakyat', 236),
 ('indonesia', 229),
 ('perintah', 227)]

In [19]:
# Hapus most freq words
FREQWORDS = set([w for (w, wc) in cnt.most_common(2)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split(' ') if word not in FREQWORDS])

df["text_wo_stopfreq"] = df["text_stemmed"].apply(lambda text: remove_freqwords(text))
df.head()

,text,sentimen,text_wo_punct,text_custom,text_rmvcustom,text_wo_stop,text_stemmed,text_wo_stopfreq
0,ya allah musnahkan lah virus corona dr muka bu...,0,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dari muk...,allah musnahkan virus corona muka bumi menja...,allah musnah virus corona muka bumi jalan hi...,allah musnah virus corona muka bumi jalan hi...
1,"sekarang saya malu , kenapa dulu terlalu memb...",-1,sekarang saya malu kenapa dulu terlalu memb...,sekarang saya malu kenapa dulu terlalu membang...,sekarang saya malu kenapa dulu terlalu membang...,sekarang malu kenapa dulu terlalu membanggakan...,sekarang malu kenapa dulu terlalu bangga belia...,sekarang malu kenapa dulu terlalu bangga belia...
2,"peraturan anehh semua ,,, aku nyaris mati kela...",-1,peraturan anehh semua aku nyaris mati kela...,peraturan aneh semua aku nyaris mati kelapan s...,peraturan aneh semua saya nyaris mati kelapa...,peraturan aneh semua nyaris mati kelapan sam...,atur aneh semua nyaris mati kelap sama anak ...,atur aneh semua nyaris mati kelap sama anak ...
3,saya selalu nunggu new normal biar saya segera...,1,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...,selalu nunggu new normal segera melangsungkan ...,selalu nunggu new normal segera langsung nikah...,selalu nunggu segera langsung nikah tunggu doa...
4,semoga covid cepat berlallu..amin🤲 \nkaltim pu...,1,semoga covid cepat berlallu amin🤲 kaltim pun...,semoga covid cepat berlallu amin 1F932 kaltim...,semoga covid cepat berlallu amin 1F932 kaltim...,semoga covid cepat berlallu amin 1F932 kaltim...,moga covid cepat berlallu amin 1f932 kaltim s...,moga covid cepat berlallu amin 1f932 kaltim s...


# Least freq words

In [20]:
from collections import Counter
cnt = Counter()
for text in df["text_stemmed"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common()[:-5-1:-1]

[('dermawan', 1), ('subsidi', 1), ('berakir', 1), ('gerah', 1), ('ekonmi', 1)]

In [21]:
# Drop the two columns which are no more needed 
# df.drop(["text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

n_rare_words = 100
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split(' ') if word not in RAREWORDS])

df["text_wo_stopfreq"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df.head()

,text,sentimen,text_wo_punct,text_custom,text_rmvcustom,text_wo_stop,text_stemmed,text_wo_stopfreq
0,ya allah musnahkan lah virus corona dr muka bu...,0,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dr muka bu...,ya allah musnahkan lah virus corona dari muk...,allah musnahkan virus corona muka bumi menja...,allah musnah virus corona muka bumi jalan hi...,allah musnah virus corona muka bumi jalan hi...
1,"sekarang saya malu , kenapa dulu terlalu memb...",-1,sekarang saya malu kenapa dulu terlalu memb...,sekarang saya malu kenapa dulu terlalu membang...,sekarang saya malu kenapa dulu terlalu membang...,sekarang malu kenapa dulu terlalu membanggakan...,sekarang malu kenapa dulu terlalu bangga belia...,sekarang malu kenapa dulu terlalu bangga belia...
2,"peraturan anehh semua ,,, aku nyaris mati kela...",-1,peraturan anehh semua aku nyaris mati kela...,peraturan aneh semua aku nyaris mati kelapan s...,peraturan aneh semua saya nyaris mati kelapa...,peraturan aneh semua nyaris mati kelapan sam...,atur aneh semua nyaris mati kelap sama anak ...,atur aneh semua nyaris mati kelap sama anak ...
3,saya selalu nunggu new normal biar saya segera...,1,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...,saya selalu nunggu new normal biar saya segera...,selalu nunggu new normal segera melangsungkan ...,selalu nunggu new normal segera langsung nikah...,selalu nunggu segera langsung nikah tunggu doa...
4,semoga covid cepat berlallu..amin🤲 \nkaltim pu...,1,semoga covid cepat berlallu amin🤲 kaltim pun...,semoga covid cepat berlallu amin 1F932 kaltim...,semoga covid cepat berlallu amin 1F932 kaltim...,semoga covid cepat berlallu amin 1F932 kaltim...,moga covid cepat berlallu amin 1f932 kaltim s...,moga covid cepat berlallu amin 1f932 kaltim s...


In [183]:
# dfbc = df.copy()
# df = pd.merge(train, test, how='outer')

In [23]:
# COL = "text_wo_emoji"
COL = 'text_wo_stopfreq'

In [24]:
print(COL)
df.shape

text_wo_stopfreq


(1460, 8)

In [196]:
# df[:len(train)]
# df[len(train):]

# TF-IDF, Bigram

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=3, max_df=4.0, ngram_range=(1, 2))
features = tfidf.fit_transform(df[COL].values)
df2 = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())

In [29]:
print(df2.shape)

(1460, 1803)


In [30]:
df2['y'] = df['sentimen']

### Split test with equal number of class,

In [31]:
# test0 = df[df.y==-1].sample(60)
# test1 = df[df.y==0].sample(60)
# test2 = df[df.y==1].sample(60)
# train=df[~df.isin(test0)&~df.isin(test1)&~df.isin(test2)].dropna()
# test = pd.merge(test0, test1, how='outer')
# test = pd.merge(test, test2, how='outer')
# test = test.sample(len(test))
# print(train.y.value_counts())
# print(test.y.value_counts())

# dftrain = df2[:len(train)]
# dftest = df2[len(train):]
# X_train = dftrain.drop(['y'], axis=1)
# y_train = dftrain['y']
# X_test = dftest.drop(['y'], axis=1)
# y_test = dftest['y']

### or split randomly

In [33]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import tree, metrics
from sklearn.metrics import classification_report

In [48]:
X_train, X_test, y_train, y_test = train_test_split(df2.drop(['y'], axis=1), df2[['y']], test_size=0.25, random_state=555)

## Logistic regression

In [50]:
from sklearn.linear_model import LogisticRegression

rgs = LogisticRegression(max_iter=600, 
                         C=1.65,
                         solver='lbfgs',
                         class_weight='balanced',
                         n_jobs=-1)

rgs.fit(X_train, y_train.values.ravel())
y_pred = rgs.predict(X_test)
print(rgs.score(X_test, y_test))
print(metrics.classification_report(y_test, y_pred, digits=3))

0.6273972602739726
              precision    recall  f1-score   support

          -1      0.671     0.723     0.696       155
           0      0.586     0.586     0.586       128
           1      0.600     0.512     0.553        82

    accuracy                          0.627       365
   macro avg      0.619     0.607     0.611       365
weighted avg      0.625     0.627     0.625       365



## SVM

In [338]:
from sklearn import svm

clf = svm.SVC(max_iter=-1, 
            C=1000,
#             gamma=0.01,
            kernel='rbf')
# clf = svm.NuSVC(kernel='poly')
# acc = cross_val_score(clf, X, Y, cv=4)
clf.fit(X_train, y_train.values.ravel())
y_pred = clf.predict(X_test)
print(clf.score(X_test, y_test))
print(metrics.classification_report(y_test, y_pred, digits=3))

0.6277777777777778
              precision    recall  f1-score   support

        -1.0      0.548     0.850     0.667        60
         0.0      0.618     0.567     0.591        60
         1.0      0.875     0.467     0.609        60

    accuracy                          0.628       180
   macro avg      0.681     0.628     0.622       180
weighted avg      0.681     0.628     0.622       180



## Multinomial NB

In [337]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
clf = MultinomialNB(alpha=0.25)
clf.fit(X_train, y_train.values.ravel())
y_pred = clf.predict(X_test)
print(clf.score(X_test, y_test))
print(metrics.classification_report(y_test, y_pred, digits=3))

0.6333333333333333
              precision    recall  f1-score   support

        -1.0      0.559     0.867     0.680        60
         0.0      0.623     0.550     0.584        60
         1.0      0.853     0.483     0.617        60

    accuracy                          0.633       180
   macro avg      0.678     0.633     0.627       180
weighted avg      0.678     0.633     0.627       180



# TF-IDF, Unigram

In [295]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.5, ngram_range=(1, 1))
features = tfidf.fit_transform(df[COL].values)
dfuni = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())

In [296]:
print(dfuni.shape)

(1460, 1218)


In [297]:
dfuni['y'] = df['sentimen']

In [298]:
dfunitrain = dfuni[:len(train)]
dfunitest = dfuni[len(train):]

In [299]:
X_unitrain = dfunitrain.drop(['y'], axis=1)
y_unitrain = dfunitrain['y']
X_unitest = dfunitest.drop(['y'], axis=1)
y_unitest = dfunitest['y']

## Logistic regression

In [335]:

rgs = LogisticRegression(max_iter=600, 
                         C=1.65,
                         solver='lbfgs',
                         class_weight='balanced',
                         n_jobs=-1)

rgs.fit(X_unitrain, y_unitrain.values.ravel())
y_unipred = rgs.predict(X_unitest)
print('Accuracy:', rgs.score(X_unitest, y_unitest))
print(metrics.classification_report(y_unitest, y_unipred, digits=3))

Accuracy: 0.6277777777777778
              precision    recall  f1-score   support

        -1.0      0.558     0.717     0.628        60
         0.0      0.653     0.533     0.587        60
         1.0      0.704     0.633     0.667        60

    accuracy                          0.628       180
   macro avg      0.638     0.628     0.627       180
weighted avg      0.638     0.628     0.627       180



## SVM

In [333]:
clf = svm.SVC(max_iter=-1, 
            C=1000,
#             gamma=0.01,
            kernel='rbf')
# clf = svm.NuSVC(kernel='poly')
clf.fit(X_unitrain, y_unitrain.values.ravel())
y_unipred = clf.predict(X_unitest)
print(clf.score(X_unitest, y_unitest))
print(metrics.classification_report(y_unitest, y_unipred, digits=3))

0.5944444444444444
              precision    recall  f1-score   support

        -1.0      0.517     0.767     0.617        60
         0.0      0.571     0.533     0.552        60
         1.0      0.829     0.483     0.611        60

    accuracy                          0.594       180
   macro avg      0.639     0.594     0.593       180
weighted avg      0.639     0.594     0.593       180



## Multinomial NB

In [336]:
clf = MultinomialNB(alpha=0.25)
clf.fit(X_unitrain, y_unitrain.values.ravel())
y_unipred = clf.predict(X_unitest)
print(clf.score(X_unitest, y_unitest))
print(metrics.classification_report(y_unitest, y_unipred, digits=3))

0.6166666666666667
              precision    recall  f1-score   support

        -1.0      0.521     0.833     0.641        60
         0.0      0.627     0.533     0.577        60
         1.0      0.879     0.483     0.624        60

    accuracy                          0.617       180
   macro avg      0.676     0.617     0.614       180
weighted avg      0.676     0.617     0.614       180



In [342]:
# pd.set_option('display.max_colwidth', 2000)